In [11]:
from openai import OpenAI
import os
from dotenv import dotenv_values
from ReadLoad import read_jsonl, write_jsonl, read_json,write_json
from tqdm import tqdm
from evaluation import get_embedding, cosine_similarity
from prompt_template import get_sys_prompt, get_qa_prompt
#from shusheng import  get_respond
from eval_data import key_word_score
from qwen import qwen_response
import pandas as pd
import time

config = dotenv_values('.env')
client = OpenAI(
    api_key=config['qwen_key'],
    base_url=config['qwen_url']
)


def get_response(prompt):
    time.sleep(0.1)
    try:
        completion = client.chat.completions.create(
            model="qwen2-7b-instruct",
            messages=[
                {'role': 'system', 'content': get_sys_prompt()},
                {'role': 'user', 'content': prompt}
            ]

            )
        return completion.choices[0].message.content
    except Exception as e:
    # Handle any other unexpected exceptions
        print(f"An unexpected error occurred: {e}")
        return f"An unexpected error occurred: {e}"


Downloading: 100%|██████████| 663/663 [00:00<00:00, 1.09kB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 74.5B/s]
Downloading: 100%|██████████| 243/243 [00:00<00:00, 494B/s]
Downloading: 100%|██████████| 11.1k/11.1k [00:00<00:00, 21.6kB/s]
Downloading: 100%|██████████| 1.59M/1.59M [00:00<00:00, 2.76MB/s]
Downloading: 100%|██████████| 3.67G/3.67G [00:09<00:00, 396MB/s] 
Downloading: 100%|██████████| 3.60G/3.60G [00:09<00:00, 389MB/s] 
Downloading: 100%|██████████| 3.60G/3.60G [00:09<00:00, 402MB/s] 
Downloading: 100%|██████████| 3.31G/3.31G [00:11<00:00, 317MB/s] 
Downloading: 100%|██████████| 27.1k/27.1k [00:00<00:00, 65.7kB/s]
Downloading: 100%|██████████| 6.41k/6.41k [00:00<00:00, 17.2kB/s]
Downloading: 100%|██████████| 6.70M/6.70M [00:00<00:00, 9.67MB/s]
Downloading: 100%|██████████| 1.26k/1.26k [00:00<00:00, 3.24kB/s]
Downloading: 100%|██████████| 2.65M/2.65M [00:00<00:00, 5.81MB/s]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def completion(data):
    for d in tqdm(data):
        query = d['问题']
        clause = d['条款']
        name = d['产品名']
        prompt = get_qa_prompt(name,clause,query)
        d['prompt'] = prompt
        #d['answer'] = get_response(prompt)
        #d['answer'] = get_respond(prompt)
        d['answer'] = qwen_response(prompt)
    return data

In [3]:
def evaluation(data, name="测试数据"):
    similaritys = []
    for d in tqdm(data):
        vec1 = dev_data_embd[d['ID']]
        vec2 = get_embedding(d['answer'])
        similarity = cosine_similarity(vec1, vec2)
        d['similarity'] = similarity
        d['key_word_similarity'] = key_word_score(d['答案'], d['answer'])
        similaritys.append(similarity)
    length = len(data)
    score = sum(similaritys)/length
    print(f'测评数据集：{length} 模型推理结果与答案的相识度得分：{score}')
    df = pd.DataFrame(data)
    df.to_excel(f"{name}_score_{score}.xlsx", index=False)
    return data

In [29]:
df = pd.DataFrame(train_eval)
df.to_excel("train_eval.xlsx", index=False)

In [4]:
def to_summit_json(data, commit="大道至简_result"):
    result = []
    for d in data:
        rd = {
            "ID": d['ID'],
            "question": d['问题'],
            "answer": d['answer']
        }
        result.append(rd)
    #write_json(result, commit)
    write_jsonl(result, commit)

### loading data

In [12]:
test_data = read_json("dataset/test.json")
# dev_data = read_jsonl("dataset/resultdev_with_embedding.jsonl")
# dev_data_embd = {}
# for d in dev_data:
#     dev_data_embd[d['ID']] = d['ans_embedding']
#     d.pop('ans_embedding')

In [6]:
# 将数据也移到GPU上
# comp_data = completion(dev_data[:100])
# eval_data = evaluation(comp_data)

test_data_result = completion(test_data)
to_summit_json(test_data_result)


100%|██████████| 1000/1000 [25:26<00:00,  1.53s/it]


In [7]:
from qwen import qwen_response
qwen_response("你好")

'你好！有什么关于保险条款的问题我可以帮助你解答吗？'

### 参考资料

https://dashscope.console.aliyun.com/billing